TITLE HERE

Intro start. explain tennis rankings, how you move up and down, is 1 good or bad etc) explain a bit about the ace serve 
and double fault. what they are

explain our question. can we estimate a players ranking based on ratio of ace serve to double fault etc


descibe data. what are we measuring

**Reading The Data**

In [ ]:
# load packages
library(tidyverse)
library(tidymodels)
library(repr)
# read the data 
# note: The code below will download the data into the current working directory and read it
url <- "https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn"
download.file(url, "./atp2017-2019.csv")
player_result <- read_csv("./atp2017-2019.csv")
glimpse(player_result)

**Cleaning and Wrangling The Data**

In [ ]:
# filter year 2019, there is indeed a more formal way with more steps to do that,
# but we can simply do it this way with only one step because we don't need the
# year information later on
player_2019 <- filter(player_result, tourney_date <= 20199999
                       , tourney_date >=20190000) |>
  select(winner_id, loser_id, w_ace, l_ace, w_df, l_df, winner_rank, loser_rank)

# seperate the original data into winner and loser
player_winner <- player_2019 |>
  select(winner_id, w_ace, w_df, winner_rank) |>
  rename(id = winner_id, ace = w_ace, df = w_df, rank = winner_rank)
player_loser <- player_2019 |>
  select(loser_id, l_ace, l_df, loser_rank) |>
  rename(id = loser_id, ace = l_ace, df=l_df, rank = loser_rank)

# Combine them together by rows to form a data into a tidy format
player_clean <- bind_rows(player_winner, player_loser)
head(player_clean)

In [ ]:
# Split data into training data and testing data
player_split <- player_clean |>
  initial_split(prop = 0.75, strata = rank)
player_training <- training(player_split)
player_testing <- testing(player_split)
head(player_training)

**Summarizing The Data**

In [ ]:
# Exploratory data analysis to training data
Summarised_training <- player_training |>
  summarize(ace_mean = mean(ace, na.rm = TRUE),
            df_mean = mean(df, na.rm = TRUE),
            missing_id = sum(is.na(id)), 
            missing_ace = sum(is.na(ace)),
            missing_df = sum(is.na(df)),
            missing_rank = sum(is.na(rank)))
Summarised_training

**Visualizing The Data**

In [ ]:
ace_plot<-ggplot(player_training, aes(x=ace))+
geom_histogram(binwidth=1, colour="black", fill="white")+
xlab("Ace serves (Per Player Per game)")
ylab("Count")+
theme(text = element_text(size=25))
df_plot<-ggplot(player_training, aes(x=df))+
geom_histogram(binwidth=1, colour="black", fill="white")+
xlab("Double Faults (Per Player Per game)")
ylab("Count")+
theme(text = element_text(size=50))
ace_plot
df_plot

WRITING SECTION describe how we are going to do the regression question. How will we visualize it ?

WRITING SECTIONOutcomes. predict results. impact. questions will it lead too